# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [197]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jbo1881\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jbo1881\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jbo1881\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [191]:
# load data from database
engine = create_engine('sqlite:///Disaster_Messages.db')
df = pd.read_sql_table('Disaster_Messages', engine)
X = df['message']
y = df.loc[:, 'related':] 

### 2. Write a tokenization function to process your text data

In [198]:
def tokenize(text):

    '''
    Tokenizes and preprocesses text data.

    Parameters:
    text (str): The input text to be tokenized and preprocessed.

    Returns:
    list of str: A list of preprocessed and lemmatized tokens.

    This function performs the following text preprocessing steps:
    1. Normalizes the input text by converting it to lowercase and removing special characters.
    2. Tokenizes the normalized text into individual words.
    3. Removes common English stopwords from the tokenized words.
    4. Lemmatizes each token to reduce words to their base form.
    '''

    # 1. Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text.lower())

    # 2. Tokenize text
    words = word_tokenize(text)

    # 3. Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]

    # 4. Lemmatization
    lemmat = [WordNetLemmatizer().lemmatize(w) for w in words]

    return lemmat



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [193]:
# Pipeline: Naive Bayes classifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [199]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the RandomizedSearchCV to training data
pipeline.fit(X_train, y_train)


C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001DCC5347D30>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [200]:
# Predict using the pipeline
y_pred = pipeline.predict(X_test)

# Iterate through each column (feature)
for i, col in enumerate(y_test.columns):
    print(f'Feature {i + 1}: {col}')
    
    # Calculate and print the classification report
    report = classification_report(y_test[col], y_pred[:, i])
    print(report)
    
# Calculate overall accuracy
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

Feature 1: related
              precision    recall  f1-score   support

           0       0.77      0.10      0.17      1501
           1       0.78      0.99      0.87      5002
           2       0.00      0.00      0.00        51

    accuracy                           0.78      6554
   macro avg       0.52      0.36      0.35      6554
weighted avg       0.77      0.78      0.71      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5456
           1       0.86      0.21      0.34      1098

    accuracy                           0.86      6554
   macro avg       0.86      0.60      0.63      6554
weighted avg       0.86      0.86      0.83      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6220
           1       0.00      0.00      0.00       334

    accuracy                           0.95      6554
   macro avg       0.47      0.50      0.49      6554
weighted avg       0.90      0.95      0.92      6554

Feature 23: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6409
           1       0.00      0.00      0.00       145

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.96      0.98      0.97      6554

Feature 24: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6513
           1       0.00      0.00      0.00        41

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.9

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 6. Improve your model
Use grid search to find better parameters. 

In [201]:
# Define the parameter grid to search over for Naive Bayes
parameters = {
    'tfidf__use_idf': [True, False],  # TF-IDF use_idf parameter
    'clf__estimator__alpha': [0.1, 0.5, 1.0]  # Alpha parameter for Multinomial Naive Bayes
}

# Create the GridSearchCV object for Naive Bayes
cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring='accuracy')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [202]:

# Fit the grid search (X_train, y_train)
cv.fit(X_train, y_train)

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001DCC5347D30>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=MultinomialNB()))]),
             param_grid={'clf__estimator__alpha': [0.1, 0.5, 1.0],
                         'tfidf__use_idf': [True, False]},
             scoring='accuracy')

In [203]:

y_pred = cv.predict(X_test)

# Iterate through each column (feature)
for i, col in enumerate(y_test.columns):
    print(f'Feature {i + 1}: {col}')
    
    # Calculate and print the classification report
    report = classification_report(y_test[col], y_pred[:, i])
    print(report)
    
# Calculate overall accuracy
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))


Feature 1: related
              precision    recall  f1-score   support

           0       0.70      0.39      0.50      1501
           1       0.83      0.95      0.89      5002
           2       0.70      0.27      0.39        51

    accuracy                           0.82      6554
   macro avg       0.75      0.54      0.60      6554
weighted avg       0.80      0.82      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      5456
           1       0.71      0.54      0.62      1098

    accuracy                           0.89      6554
   macro avg       0.81      0.75      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6112
           1       0.60      0.10      0.17       442

    accuracy                           0.93      6554
   macro avg       0.77      0.55      0.57      6554
weighted avg       0.92      0.93      0.91      6554

Feature 12: food
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5816
           1       0.71      0.30      0.42       738

    accuracy                           0.91      6554
   macro avg       0.81      0.64      0.69      6554
weighted avg       0.89      0.91      0.89      6554

Feature 13: shelter
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      5948
           1       0.65      0.18      0.29       606

    accuracy                           0.92      6554
   macro avg       0.79      0.59      0.62      6554
weighted avg       0.90      0.92    

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [185]:
## Random Forest Classifier

# Pipeline:
RF_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

In [186]:
# Fit the Random Forest Classifier 
RF_pipeline.fit(X_train, y_train)

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001DCE6496D30>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [187]:
# Predict using the Random Forest pipeline
y_pred_RF = RF_pipeline.predict(X_test)

# Iterate through each column (feature)
for i, col in enumerate(y_test.columns):
    print(f'Feature {i + 1}: {col}')
    
    # Calculate and print the classification report
    report_RF = classification_report(y_test[col], y_pred_RF[:, i])
    print(report_RF)
    
# Calculate overall accuracy
accuracy_RF = (y_pred_RF == y_test.values).mean()
print('The Random Forest model accuracy is {:.3f}'.format(accuracy_RF))

Feature 1: related
              precision    recall  f1-score   support

           0       0.74      0.44      0.55      1553
           1       0.84      0.95      0.89      4955
           2       0.29      0.37      0.33        46

    accuracy                           0.82      6554
   macro avg       0.62      0.58      0.59      6554
weighted avg       0.81      0.82      0.81      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5442
           1       0.86      0.50      0.63      1112

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.90      0.90      0.89      6554

Feature 3: offer


C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Feature 4: aid_related
              precision    recall  f1-score   support

           0       0.81      0.84      0.82      3867
           1       0.76      0.72      0.74      2687

    accuracy                           0.79      6554
   macro avg       0.78      0.78      0.78      6554
weighted avg       0.79      0.79      0.79      6554

Feature 5: medical_help
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6044
           1       0.62      0.09      0.15       510

    accuracy                           0.92      6554
   macro avg       0.77      0.54      0.56      6554
weighted avg       0.90    

C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jbo1881\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6263
           1       0.00      0.00      0.00       291

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.96      0.93      6554

Feature 29: weather_related
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      4738
           1       0.85      0.71      0.77      1816

    accuracy                           0.88      6554
   macro avg       0.87      0.83      0.85      6554
weighted avg       0.88      0.88      0.88      6554

Feature 30: floods
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5993
           1       0.88      0.46      0.60       561

    accuracy                           0.95      6554
   macro avg       0.92      0.73      0.79      6554
weighted avg       0.95    

In [188]:
# Define the parameter grid for Random Forest Classifier
parameters_rf = {
    'tfidf__use_idf': [True, False],  # TF-IDF use_idf parameter
    'clf__estimator__n_estimators': [10, 20, 50],  # Number of trees in the forest
    'clf__estimator__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'clf__estimator__min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
}

# Create the GridSearchCV object for Random Forest Classifier
cv_rf = GridSearchCV(RF_pipeline, param_grid=parameters_rf, cv=5, scoring='accuracy')

In [ ]:
# Fit the grid search (X_train, y_train)
cv_rf.fit(X_train, y_train)

In [ ]:
y_pred_cvrf = cv_rf.predict(X_test)

# Iterate through each column (feature)
for i, col in enumerate(y_test.columns):
    print(f'Feature {i + 1}: {col}')
    
    # Calculate and print the classification report
    report_cvrf = classification_report(y_test[col], y_pred_cvrf[:, i])
    print(report_cvrf)
    
# Calculate overall accuracy
accuracy_cvrf = (y_pred_cvrf == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy_cvrf))

### 9. Export your model as a pickle file

In [208]:
# Save the model to a pickle file (Naive Bayes classifier)
import pickle

with open('classifier.pkl', 'wb') as model_file:
    pickle.dump(cv, model_file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.